In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df= pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [ ]:
train_df.head(3)

In [ ]:
train_df.info()

In [ ]:
l = train_df.isnull().sum()
print(l[l>0])

In [ ]:
train_df = train_df.dropna(subset=['Electrical','MasVnrType','MasVnrArea'])

In [ ]:
train_df['PoolQCpresent']=train_df['PoolQC'].isnull()

In [ ]:
import seaborn as sns

In [ ]:
sns.swarmplot(train_df.PoolQCpresent, train_df.SalePrice)

In [ ]:
train_df = train_df.drop(columns=['PoolQCpresent','PoolQC'])

In [ ]:
s = train_df.dtypes == 'object'
cols = list(s[s].index)
print(cols)

In [ ]:
sns.swarmplot(train_df.MSZoning, train_df.SalePrice)

In [ ]:
sns.swarmplot(train_df.Street, train_df.SalePrice)

In [ ]:
sns.swarmplot(train_df.Alley, train_df.SalePrice)

In [ ]:
sns.swarmplot(train_df.LotShape, train_df.SalePrice)

In [ ]:
sns.swarmplot(train_df.Fireplaces, train_df.SalePrice)

In [ ]:
from sklearn.impute import SimpleImputer

si = SimpleImputer()
imputed_df = pd.DataFrame(si.fit_transform(train_df))

In [ ]:
train_df = train_df.fillna(0)


In [ ]:
from sklearn.preprocessing import LabelEncoder
# label encoding
for column in train_df.columns:
    if train_df[column].dtype == 'O':  # if it's not Object dtype
        le = LabelEncoder() 
        train_df[column] = le.fit_transform(train_df[column].astype('str'))

In [ ]:
imputed_df.head(5)

In [ ]:
cols =[i for i in train_df.columns if train_df.dtypes[i]!='object' and not(train_df[i].isnull().any())]
print(cols)

In [ ]:
from sklearn.feature_selection import mutual_info_regression
y = train_df.SalePrice.values
X = train_df[cols]
mi = mutual_info_regression(X, y)
mi /= np.max(mi)
mi =pd.Series(mi, name="MI Scores", index=cols)
mi.sort_values()


In [ ]:
train_df = train_df.drop(columns=['MoSold','MiscVal','PoolArea','3SsnPorch','LowQualFinSF','BsmtFinSF2','YrSold','KitchenAbvGr','BsmtHalfBath','BsmtFullBath','ScreenPorch','EnclosedPorch','BedroomAbvGr','HalfBath','MasVnrArea','WoodDeckSF'])

In [ ]:
train_df = train_df.drop(columns=['MSSubClass','OpenPorchSF','BsmtUnfSF','LotArea','2ndFlrSF'])

In [ ]:
train_df = train_df.drop(columns=['Alley','MiscFeature','Fence'])

In [ ]:
train_df = train_df.drop(columns=['Fireplaces'])

In [ ]:
train_df['Street'] = train_df['Street'].map({'Grvl': 1, 'Pave': 0})

In [ ]:
train_df = train_df.drop(columns=['Street'])

In [ ]:
sns.scatterplot(train_df.MSSubClass, train_df.SalePrice)

In [ ]:
sns.scatterplot(train_df.OpenPorchSF, train_df.SalePrice)

In [ ]:
train_df = train_df.drop(columns=['OpenPorchSF'])

In [ ]:
from sklearn.feature_selection import mutual_info_regression
y = train_df.SalePrice.values
X = train_df[cols]
mi = mutual_info_regression(X, y)
mi /= np.max(mi)
mi =pd.Series(mi, name="MI Scores", index=cols)
mi.sort_values()


In [ ]:
train_df = train_df.drop(columns = ['Functional','Condition2','RoofStyle','Utilities','RoofMatl','LotConfig','BsmtFinType2','Condition1','LandContour','LandSlope','Heating'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()



In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
model =XGBRegressor(n_estimators=50)

X = train_df.drop("SalePrice", axis=1).values
X_sc = scaler.fit_transform(X)
y = train_df.SalePrice.values
X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
from sklearn.metrics import mean_squared_error
predY = model.predict(X_test)
np.sqrt(mean_squared_error(y_test, predY))

In [ ]:
test_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
test_df = test_df.drop(columns=['PoolQC','MoSold','MiscVal','PoolArea','3SsnPorch','LowQualFinSF','BsmtFinSF2','YrSold','KitchenAbvGr','BsmtHalfBath','BsmtFullBath','ScreenPorch','EnclosedPorch','BedroomAbvGr','HalfBath','MasVnrArea','WoodDeckSF','MSSubClass','OpenPorchSF','BsmtUnfSF','LotArea','2ndFlrSF','Alley','MiscFeature','Fence','Fireplaces','Street','Alley','MiscFeature','Fence','Functional','Condition2','RoofStyle','Utilities','RoofMatl','LotConfig','BsmtFinType2','Condition1','LandContour','LandSlope','Heating'])

In [ ]:
test_df2 = test_df.fillna(0)

for column in test_df2.columns:
    if column == 'Id':
        continue
    if test_df2[column].dtype == 'O':  # if it's not Object dtype
        le = LabelEncoder()  # label encoding
        test_df2[column] = le.fit_transform(test_df2[column].astype('str'))

In [ ]:
predY = model.predict(test_df2)

In [ ]:
Id = test_df.Id.values
final_df = pd.DataFrame({"Id" : Id, "SalePrice" : predY})

In [ ]:
final_df.head(20)

In [ ]:
final_df.to_csv('submission.csv',index=False)